In [3]:
from collections import defaultdict
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import norm
import sys

In [78]:
class CensusTractForecast:
    
    def __init__(self, market_area, last_full_month, sales_count_threshold=10, alpha_level=0.1):
        self.listings_df = pd.read_csv(f'{market_area}_listing_dates_with_ct.csv')
        self.ct_stats_df = pd.read_csv(f'{market_area}_census_tract_data_updated.csv')
        self.sales_count_threshold = sales_count_threshold
        self.last_full_month = last_full_month
        self.alpha_level = alpha_level
        self.ct_activity_df = self._get_ct_activity_df()
    
    def _get_ct_activity_df(self):
        listings_df = self.listings_df
        listings_df['ct_key'] = listings_df['ct_key'].astype('int')
        listings_df['list_month'] = (pd.to_datetime(listings_df['list_date']) + 
                                     pd.offsets.MonthBegin(n=1) - pd.offsets.MonthBegin(n=1))
        listings_df['sale_month'] = (pd.to_datetime(listings_df['sale_date']) + 
                                     pd.offsets.MonthBegin(n=1) - pd.offsets.MonthBegin(n=1))
        
        num_listings_ct = (listings_df.dropna(subset=['list_month'])
                   .groupby(['ct_key', 'list_month']).count()[['property_id']].reset_index())
        num_listings_ct = num_listings_ct.rename(columns={'property_id':'ct_count_listings', 
                                                          'list_month':'month'})

        num_sales_ct = (listings_df.dropna(subset=['sale_month'])
                        .groupby(['ct_key', 'sale_month']).count()[['property_id']].reset_index())
        num_sales_ct = num_sales_ct.rename(columns={'property_id':'ct_count_sales', 'sale_month':'month'})

        num_sales_listings_df = pd.merge(num_listings_ct, num_sales_ct, left_on=['month','ct_key'], 
                                         right_on=['month','ct_key'], how='outer')
        num_sales_listings_df = num_sales_listings_df.fillna(0.)
        
        ct_stats_df = self.ct_stats_df
        ct_stats_df = ct_stats_df[['ct_key', 'total_households']]
        
        ct_activity_df = pd.merge(ct_stats_df, num_sales_listings_df, left_on='ct_key', right_on='ct_key')
        
        listing_sales_overall = num_sales_listings_df.groupby(['ct_key','month']).agg(
            {'ct_count_listings':'mean', 'ct_count_sales':'mean'}).reset_index()
        listing_sales_overall = listing_sales_overall.groupby('month').agg(
            {'ct_count_listings':'sum', 'ct_count_sales':'sum'})

        # sum up number of households for all CT in the house listing dataset and the CT-level feature dataset
        listing_sales_overall['total_households'] = ct_stats_df[ct_stats_df['ct_key'].isin(
            num_sales_listings_df.ct_key.unique())]['total_households'].sum()

        listing_sales_overall['sales_per_households'] = (listing_sales_overall['ct_count_sales'] /
                                                         listing_sales_overall['total_households'])
        listing_sales_overall['listings_per_households'] = (listing_sales_overall['ct_count_listings'] /
                                                            listing_sales_overall['total_households'])
        
        ct_activity_df = pd.merge(ct_activity_df, 
                             listing_sales_overall[['sales_per_households', 'listings_per_households']], 
                             left_on='month', right_index=True)
        ct_activity_df['ct_listings_per_households'] = (ct_activity_df['ct_count_listings'] / 
                                                        ct_activity_df['total_households'])
        ct_activity_df['ct_sales_per_households'] = (ct_activity_df['ct_count_sales'] / 
                                                     ct_activity_df['total_households'])

        ct_activity_df['relative_listings_toBaseline'] = (ct_activity_df['ct_listings_per_households'] / 
                                                     ct_activity_df['listings_per_households'])
        ct_activity_df['relative_sales_toBaseline'] = (ct_activity_df['ct_sales_per_households'] / 
                                                  ct_activity_df['sales_per_households'])
        
        droppped_ct = ct_activity_df.groupby('ct_key').count()['ct_count_sales'].reset_index(name='count')
        droppped_ct = droppped_ct[droppped_ct['count'] < self.sales_count_threshold]['ct_key'].values
        
        ct_activity_df = ct_activity_df[~ct_activity_df['ct_key'].isin(droppped_ct)]
        ct_activity_df = ct_activity_df[ct_activity_df['month'] <= pd.Timestamp(self.last_full_month)]
        
        return ct_activity_df
    
    def get_ct_forecast(self, forecasted_listings_rate, forecasted_sales_rate):
        ratio_param_df = self.ct_activity_df.groupby('ct_key').agg(
            {'relative_listings_toBaseline': [np.mean, np.std],
             'relative_sales_toBaseline': [np.mean, np.std]})
        
        ratio_param_df.columns = ['listings_mean', 'listings_std', 'sales_mean', 'sales_std']
        
        std_multiple = abs(norm.ppf(self.alpha_level / 2))
        
        forecast_df = pd.merge(ratio_param_df, self.ct_stats_df[['ct_key', 'total_households']], 
                               left_index=True, right_on = 'ct_key', how='inner').set_index('ct_key')
        
        forecast_df['mean_listings_forecast'] = (forecasted_listings_rate * forecast_df['listings_mean'] *
                                                 forecast_df['total_households'])
        listings_lower_bound_ratio = np.maximum(
            0.0, forecast_df['listings_mean'] - std_multiple * forecast_df['listings_std'])
        listings_upper_bound_ratio = forecast_df['listings_mean'] + std_multiple * forecast_df['listings_std']
        forecast_df['lower_bound_listings_forecast'] = (
            forecasted_listings_rate * listings_lower_bound_ratio * forecast_df['total_households'])
        forecast_df['upper_bound_listings_forecast'] = (
            forecasted_listings_rate * listings_upper_bound_ratio * forecast_df['total_households'])
        
        forecast_df['mean_sales_forecast'] = (forecasted_sales_rate * forecast_df['sales_mean'] *
                                                 forecast_df['total_households'])
        sales_lower_bound_ratio = np.maximum(
            0.0, forecast_df['sales_mean'] - std_multiple * forecast_df['sales_std'])
        sales_upper_bound_ratio = forecast_df['sales_mean'] + std_multiple * forecast_df['sales_std']
        forecast_df['lower_bound_sales_forecast'] = (
            forecasted_sales_rate * sales_lower_bound_ratio * forecast_df['total_households'])
        forecast_df['upper_bound_sales_forecast'] = (
            forecasted_sales_rate * sales_upper_bound_ratio * forecast_df['total_households'])
        
        columns_to_keep = ['mean_listings_forecast', 'lower_bound_listings_forecast', 
                           'upper_bound_listings_forecast', 'mean_sales_forecast',
                           'lower_bound_sales_forecast', 'upper_bound_sales_forecast']
        
        forecast_df = forecast_df[columns_to_keep]
        return forecast_df
              

In [79]:
forecast = Forecast('denver', datetime.date(2020, 10, 1))

In [80]:
forecast_df = forecast.get_ct_forecast(0.005, 0.002)